In [13]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, glob, os, scipy.stats as stats
from neo.rawio import BlackrockRawIO
from neo.io import BlackrockIO


In [14]:
# load data
io_pt2    = BlackrockIO(filename='../results/20250717/20250717-143139-001')
block_pt2 = io_pt2.read_block(lazy=False)
seg_pt2   = block_pt2.segments[0]


In [ ]:
subj_spikes = seg_pt2.spiketrains
len(subj_spikes), subj_spikes[0]

(32,
 SpikeTrain containing 550 spikes; units s; datatype float64 
 name: 'ch97#0'
 description: 'SpikeTrain channel_id: 97, unit_id: 0'
 annotations: {'id': np.str_('Unit 97000'),
   'channel_id': 97,
   'unit_id': 0,
   'unit_tag': 'unclassified'}
 time: 0.0034 s to 1567.5468666666668 s)

In [25]:
trials_df = pd.read_csv('../results/asymmetry_final_subj-202511_difficulty-patients_order-B_2025-07-17_14h32.12.588.csv')
trials_df

,thisN,thisTrialN,thisRepN,blockN,run,condition,trial_key,stim_file_pos,stim_pos,noise_pos,...,submit_resp.stopped,subj,difficulty,sess_type,date,expName,psychopyVersion,frameRate,expStart,Unnamed: 119
0,0.0,0.0,0.0,1.0,1.0,baseline,3.0,0.3,-0.34,1.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
1,1.0,1.0,0.0,1.0,1.0,baseline,10.0,1.1,-0.18,1.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
2,2.0,2.0,0.0,1.0,1.0,baseline,33.0,3.3,0.26,3.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
3,3.0,3.0,0.0,1.0,1.0,baseline,4.0,0.5,-0.30,1.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
4,4.0,4.0,0.0,1.0,1.0,baseline,38.0,3.9,0.38,3.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,36.0,36.0,0.0,6.0,2.0,flat_comp,222.0,3.1,0.22,1.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
237,37.0,37.0,0.0,6.0,2.0,flat_comp,206.0,1.7,-0.06,3.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
238,38.0,38.0,0.0,6.0,2.0,flat_comp,203.0,1.3,-0.14,3.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
239,39.0,39.0,0.0,6.0,2.0,flat_comp,200.0,1.1,-0.18,3.0,...,NaN,202511.0,patients,B,2025-07-17_14h32.12.588,asymmetry_final,2024.2.4,60.0,2025-07-17 14h32.38.569890 -0600,NaN
